In [ ]:
#importing modules
import tweepy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re


from langdetect import detect_langs
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer



In [ ]:
# #twitter api keys
# auth = tweepy.OAuth1UserHandler(
#    )

# api = tweepy.API(auth, wait_on_rate_limit=True)


In [ ]:
# search_words = "#wakandaforever OR #WakandaForever OR #blackpanther OR #WakandaForeverID OR #BlackPantherWakandaForever OR #BlackPanther OR #BlackPanther2"
# search_query = search_words + " -filter:retweets AND -filter:replies"


# limit = 100000
# tweets = tweepy.Cursor(api.search_tweets, q=search_query, tweet_mode='extended').items(limit)

# data=[]
# for tweet in tweets:
#     data.append([tweet.id,tweet.user.screen_name, tweet.created_at, tweet.full_text,tweet.user.location, tweet.retweet_count,tweet.favorite_count, tweet.user.followers_count, tweet.user.friends_count, tweet.user.verified, tweet.source])

In [ ]:
# columns =['id','username','time_of_tweet', 'tweet', 'location', 'retweets', 'likes', 'followers', 'following', 'verified', 'tweet_source']
# bp_tweets=pd.DataFrame(data, columns=columns)
# bp_tweets.to_csv("BlackPanther-24-11.csv")

In [ ]:
#save scrapped data to csv 
tweets = pd.read_csv('BlackPanther-24-11.csv')
tweets.sample(10)

In [ ]:
#column names and datatype
tweets.info()

In [ ]:
#statistical description of the dataset
tweets.describe()

In [ ]:
#drop unnamed column
tweets.drop(columns='Unnamed: 0', inplace = True)

#fill in missing location with 'None'
tweets['location'].fillna('None', inplace=True)

#change the datatype of tweet_time to datetime 
tweets['time_of_tweet'] = tweets['time_of_tweet'].astype('datetime64')

In [ ]:
def get_hash(tweet):
    '''This function collect all the hashtags in the tweet column'''
    tweet = re.findall(r'\#\w+', tweet)
    return " ".join(tweet)

In [ ]:
#applying the get_hash function
tweets['hashtags'] = tweets['tweet'].apply(get_hash)


In [ ]:
#count the occurrence of each hashtags in the hashtags columns
hashtags = pd.Series(" ".join(tweets['hashtags']).split()).value_counts()
#hashtags.to_csv('hashtag_freq.csv')    ##save the output as a csv
hashtags[:10]

In [ ]:
def get_languages(tweet):
    '''This function detects the language in each 
    tweet and returns it in ISO 639-1 codes'''
    try:
        tweet = detect_langs(tweet)
    except: 
        tweet = "nolang"
    tweet = str(tweet).split(':')[0][1:]
    return tweet

In [ ]:
#applying the get_languages function
tweets['language'] = tweets['tweet'].apply(get_languages)

In [ ]:
#substituting the short form language names for the long names
lang_code = pd.read_csv('language_codes.csv')
dict_lang_code = dict(zip(lang_code['alpha2'], lang_code['English']))
tweets['language'] = tweets['language'].replace(dict_lang_code)

In [ ]:
#checking for changes
tweets.language.value_counts()[:10]

In [ ]:
def get_links(tweets):
    '''This function iterate over the tweets and return 
    'Yes' is there's a link and 'No' if there's no link '''
    link = re.findall(r'((http|ftp|https):\/\/)(([\w.-]*)\.([\w]*))', tweets)
    if len(link) < 1:
        return 'No'
    else: return 'Yes'


In [ ]:
#applying the get_links function
tweets['contains_media'] = tweets['tweet'].apply(get_links)

In [ ]:
#checking for changes
tweets.contains_media.value_counts()

In [2]:
cast = ['namor', 'shuri', 'ironheart', 'ramonda', 'namora', 
        'okoye', 'aneka', 'nakia', 'mbaku', 'attuma', 'everett', ]


def get_cast(tweets):
        '''This function extracts names of the cast into a new column'''      
        tweets = tweets.lower()
        tokens = word_tokenize(tweets)
        cast_names = [name for name in tokens if name in cast]
        cast_names_title = [name.title() for name in cast_names]
        return " ".join(cast_names_title)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 3)

In [ ]:
#applying the get_cast function
tweets['cast_name'] = tweets['tweet'].apply(get_cast)

In [ ]:
#count the occurrence of each name in the cast_name columns
names = pd.Series(" ".join(tweets['cast_name']).split()).value_counts()
# names.to_csv('names_freq.csv') ##save the generated dataframe
names[:10]

In [ ]:
#replace the '\n' characters with space
tweets['tweet'] = tweets['tweet'].replace('\n', ' ', regex=True)


In [ ]:
#get the sentiment scores using Sentiment Intensity Analyzer
sia = SentimentIntensityAnalyzer()
tweets['sentient_scores'] = tweets['tweet'].apply(sia.polarity_scores)
tweets['sentient_scores'] = tweets['sentient_scores'].apply(lambda x: x['compound'])

In [ ]:
def get_sentiment(tweet):
    '''This functions returns Positive, Negative or 
    Neutral depending on the state of the sentiment score'''
    if tweet > 0:
        return "Positive"
    elif tweet == 0:
        return "Neutral"
    else: return "Negative"

In [ ]:
#applying the get_sentiment function
tweets['sentiment'] = tweets['sentient_scores'].apply(get_sentiment)

In [ ]:
#checking for changes
tweets['sentiment'].value_counts()

In [ ]:
def get_country(inc_c, case, cor_c):
    '''This function substitutes the current text for the '''
    tweets.loc[tweets['c_location'].str.contains(inc_c, case=case), 'c_location'] = cor_c


In [ ]:
tweets['c_location'] = tweets['location']


dicts = {'brazil':'Brazil', 'brasil':'Brazil', 'nigeria':'Nigeria', 'méxico': 'Mexico', 'france':'France', 'india':'India', 'london':'United Kingdom', 'UK':'United Kingdom', 'england':'United Kingdom','Bonaire':'Bonaire Islands', 'España':'Spain', 'thailand':'Thailand', 'Ohio':'United States', 'South Africa':'South Africa'}
for key, val in dicts.items():
    get_country(key, False, val)


list_us = ['USA', 'KS', 'FL', 'CA', 'GA', 'AZ', 'CO','IL', 'TX', 'PA', 'DC', 'TN', 'NY', 'WA', 'NYC', 'NC', 'NV', 'OH', 'LA', 'MA', 'Los Angeles' 'new york','New York', 'Los Angeles']
for states in list_us:
    get_country(states, True, 'United States')

tweets['c_location'].replace('None', np.nan, inplace=True)
tweets['c_location'].value_counts()[:10]

In [ ]:
tweets['tweet_source'].value_counts()[:10]

In [ ]:
tweets['verified'].value_counts()

In [ ]:
tweets.sample(20)

In [ ]:
tweets.to_csv('BlackPanther_Cleaned.csv')